In [1]:
# based on https://github.com/theRealSuperMario/supermariopy/blob/master/scripts/tflogs2pandas.py

In [2]:
import tensorflow as tf

In [3]:
# import glob

In [4]:
# import os

In [5]:
import pandas as pd

In [6]:
# import traceback

In [7]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [11]:
def get_metrics(model_dir, m, hidden, layers, no_executions):
    
    DEFAULT_SIZE_GUIDANCE = {
    "compressedHistograms": 1,
    "images": 1,
    "scalars": 0,  # 0 means load all
    "histograms": 1,
    }
    
    _ROOT = '/home/developer/gcp'
    # _ROOT = 'gs:/'
    _STEM = 'cbidmltsf/dplstm'
    _MODEL = model_dir
    
    _PATHS = list()
    # how many experiments to collect?
    for row in range(no_executions):
        _PATHS.append('{}/{}/{}_{:02d}'.format(_ROOT, _STEM, _MODEL, row))
    
    # create an empty Dataframe with column names only
    metrics_df = pd.DataFrame(columns=['id', 'exp',
                                       'm_hour', 'm_day', 'm_week',
                                       'hidden_hour', 'hidden_day', 'hidden_week',
                                       'layers_hour', 'layers_day', 'layers_week',
                                       'train_metric', 'train_value', 'train_step',
                                       'eval_metric', 'eval_value', 'eval_step',
                                       'predict_mse', 'predict_mae'])

    for _PATH in _PATHS:
        
        # it only works if id_exp are in the shape 'TPU_NN_XX'!!!
        id = _PATH[-9:-3]
        exp = int(_PATH[-2:])
        
        # m_hourly = int(_PATH[-30:-28])
        # m_daily = int(_PATH[-23:-21])
        # m_weekly = int(_PATH[-16:-14])
        # hidden_hourly = int(_PATH[-28:-26])
        # hidden_daily = int(_PATH[-21:-19])
        # hidden_weekly = int(_PATH[-14:-12])
        # layers_hourly = int(_PATH[-26:-24])
        # layers_daily = int(_PATH[-19:-17])
        # layers_weekly = int(_PATH[-12:-10])

        m_hourly = m[0]
        m_daily = m[1]
        m_weekly = m[2]
        
        hidden_hourly = hidden[0]
        hidden_daily = hidden[1]
        hidden_weekly = hidden[2]
        
        layers_hourly = layers[0]
        layers_daily = layers[1]
        layers_weekly = layers[2]

        r = dict()

        # r['id'] = [id] # use here an iterable to avoid setting up an index
        # iterable is not needed anymore
        r['id'] = id
        r['exp'] = exp

        r['m_hour'] = m_hourly
        r['m_day'] = m_daily
        r['m_week'] = m_weekly

        r['hidden_hour'] = hidden_hourly
        r['hidden_day'] = hidden_daily
        r['hidden_week'] = hidden_weekly

        r['layers_hour'] = layers_hourly
        r['layers_day'] = layers_daily
        r['layers_week'] = layers_weekly

        event_acc = EventAccumulator(_PATH, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]

        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            # r = {"metric": [tag] * len(step), "value": values, "step": step}
            r['train_metric'] = 'train_{}'.format(tag)
            r['train_value'] = values[-1]
            r['train_step'] = step[-1]

        _EVAL_PATH = '{}/eval'.format(_PATH)

        event_acc = EventAccumulator(_EVAL_PATH, DEFAULT_SIZE_GUIDANCE)
        event_acc.Reload()

        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            # r = {"metric": ['eval_{}'.format(tag)] * len(step), "value": values, "step": step}
            r['eval_metric'] = 'eval_{}'.format(tag)
            r['eval_value'] = values[-1]
            r['eval_step'] = step[-1]

        metrics_df = metrics_df.append(r, ignore_index=True)
        
    return metrics_df


In [12]:
metrics = dict()

In [13]:
metrics['TPU_27'] = get_metrics('0812804_0812804_046402_TPU_27',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2],
                                10)

W0318 10:21:27.603041 140231943169792 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0318 10:21:27.612183 140231943169792 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0318 10:21:27.666004 140231943169792 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0318 10:21:27.669893 140231943169792 event_accumulator.py:352] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0318 10:21:27.719326 140231943169792 event_accumulator.py:344] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the gr

In [12]:
metrics['TPU_17'] = get_metrics('086401_083201_043201_TPU_17',
                               [8, 4, 4],
                               [64, 32, 32],
                               [4, 2, 1],
                               10)

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [12]:
metrics['TPU_16'] = get_metrics('086403_083202_043201_TPU_16')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [13]:
metrics['TPU_11'] = get_metrics('086404_043202_043201_TPU_11')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [14]:
metrics['TPU_13'] = get_metrics('086404_043202_043201_TPU_13')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [15]:
metrics['TPU_14'] = get_metrics('086404_086402_043201_TPU_14')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [16]:
metrics['TPU_15'] = get_metrics('086404_086402_043201_TPU_15')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [27]:
metrics['TPU_18'] = get_metrics('086404_086404_043202_TPU_18')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [30]:
metrics['TPU_19'] = get_metrics('086404_086404_043202_TPU_19')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [32]:
metrics['TPU_20'] = get_metrics('086404_086404_043202_TPU_20')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [35]:
metrics['TPU_21'] = get_metrics('086404_086404_046404_TPU_21')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [11]:
metrics['TPU_22'] = get_metrics('086404_086404_043202_TPU_22')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [10]:
metrics['TPU_23'] = get_metrics('086404_086404_043202_TPU_23')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [14]:
metrics['TPU_24'] = get_metrics('086404_086404_043202_TPU_24')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [16]:
metrics['TPU_25'] = get_metrics('089604_089604_043202_TPU_25')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [18]:
metrics['TPU_26'] = get_metrics('084804_084804_043202_TPU_26')

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [23]:
metrics['TPU_27'] = get_metrics('0812804_0812804_046402_TPU_27',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [32]:
metrics['TPU_30'] = get_metrics('0812804_0812804_046402_TPU_30',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [10]:
metrics['TPU_31'] = get_metrics('0812804_0812804_046402_TPU_31',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [14]:
metrics['TPU_32'] = get_metrics('0812804_0812804_046402_TPU_32',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [16]:
metrics['TPU_33'] = get_metrics('0812804_0812804_046402_TPU_33',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [18]:
metrics['TPU_34'] = get_metrics('0812804_0812804_046402_TPU_34',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [20]:
metrics['TPU_35'] = get_metrics('0812804_0812804_046402_TPU_35',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [13]:
metrics['TPU_36'] = get_metrics('0812804_0812804_046402_TPU_36',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [16]:
metrics['TPU_37'] = get_metrics('0812804_0812804_046402_TPU_37',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [19]:
metrics['TPU_38'] = get_metrics('0812804_0812804_046402_TPU_38',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [21]:
metrics['TPU_39'] = get_metrics('0812804_0812804_046402_TPU_39',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [28]:
metrics['TPU_40'] = get_metrics('0812804_0812804_046402_TPU_40',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [34]:
metrics['TPU_41'] = get_metrics('0812804_0812804_046402_TPU_41',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [36]:
metrics['TPU_42'] = get_metrics('0812804_0812804_046402_TPU_42',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2])

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [9]:
metrics['TPU_50'] = get_metrics('0812804_0812804_046402_TPU_50',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2],
                                30)

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [10]:
metrics['TPU_51'] = get_metrics('0812804_0812804_046402_TPU_51',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2],
                                30)

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [12]:
metrics['TPU_52'] = get_metrics('0812804_0812804_046402_TPU_52',
                                [8, 8, 4],
                                [128, 128, 64],
                                [4, 4, 2],
                                20)

Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
Found more than one metagr

In [40]:
parameters = dict()

In [41]:
_PROCESS_ID = 'TPU_10'
chromosome = [[16, 8, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.005, 16000, 10000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '166404_083202_043201_TPU_10',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [45]:
parameters['TPU_10']['dense']

[64, 32, 16, 8]

In [14]:
_PROCESS_ID = 'TPU_11'
chromosome = [[8, 4, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.005, 16000, 10000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_043202_043201_TPU_11',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [16]:
_PROCESS_ID = 'TPU_13'
chromosome = [[8, 4, 4],
              [64, 32, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_043202_043201_TPU_13',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [17]:
_PROCESS_ID = 'TPU_14'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086402_043201_TPU_14',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [ ]:
_PROCESS_ID = 'TPU_15'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 2, 1],
              False,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086402_043201_TPU_15',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [18]:
_PROCESS_ID = 'TPU_16'
chromosome = [[8, 8, 4],
              [64, 32, 32],
              [3, 2, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086403_083202_043201_TPU_16',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [19]:
_PROCESS_ID = 'TPU_17'
chromosome = [[8, 8, 4],
              [64, 32, 32],
              [1, 1, 1],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086401_083201_043201_TPU_17',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [20]:
_PROCESS_ID = 'TPU_18'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [64, 32, 16, 8],
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_18',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [21]:
_PROCESS_ID = 'TPU_19'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [64, 32, 16, 8],
              [0.004, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_19',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [22]:
_PROCESS_ID = 'TPU_20'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_20',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [23]:
_PROCESS_ID = 'TPU_21'
chromosome = [[8, 8, 4],
              [64, 64, 64],
              [4, 4, 4],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_046404_TPU_21',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [24]:
_PROCESS_ID = 'TPU_22'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 64],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_22',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [25]:
_PROCESS_ID = 'TPU_23'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 256, 64],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_23',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [26]:
_PROCESS_ID = 'TPU_24'
chromosome = [[8, 8, 4],
              [64, 64, 32],
              [4, 4, 2],
              True,
              [256, 256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '086404_086404_043202_TPU_24',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [27]:
_PROCESS_ID = 'TPU_25'
chromosome = [[8, 8, 4],
              [96, 96, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '089604_089604_043202_TPU_25',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [28]:
_PROCESS_ID = 'TPU_26'
chromosome = [[8, 8, 4],
              [48, 48, 32],
              [4, 4, 2],
              True,
              [256, 64, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '084804_084804_043202_TPU_26',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [29]:
_PROCESS_ID = 'TPU_27'
chromosome = [[8, 8, 4],
              [128, 128, 64],
              [4, 4, 2],
              True,
              [512, 128, 8],  # changes here are required to be hard-coded in model.py
              [0.008, 16000, 20000, 32]]

parameters[_PROCESS_ID] = {
    'model_stem': '0812804_0812804_046402_TPU_27',
    'm': chromosome[0],
    'hourly': [chromosome[1][0]]*chromosome[2][0],    
    'daily': [chromosome[1][1]]*chromosome[2][1],    
    'weekly': [chromosome[1][2]]*chromosome[2][2],    
    'use_timestamps': chromosome[3],
    'dense': chromosome[4],
    'train_specs': chromosome[5],
    'no_executions': 10
}

In [11]:
metrics['TPU_10'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000547,0.000588,NaN,NaN
std,0.000657,0.000412,NaN,NaN
min,0.000059,0.000290,NaN,NaN
25%,0.000214,0.000394,NaN,NaN
50%,0.000265,0.000481,NaN,NaN
75%,0.000348,0.000555,NaN,NaN
max,0.001937,0.001728,NaN,NaN


In [18]:
metrics['TPU_11'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000407,0.000528,NaN,NaN
std,0.000285,0.000098,NaN,NaN
min,0.000160,0.000403,NaN,NaN
25%,0.000192,0.000452,NaN,NaN
50%,0.000333,0.000530,NaN,NaN
75%,0.000511,0.000579,NaN,NaN
max,0.001060,0.000712,NaN,NaN


In [28]:
metrics['TPU_13'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.005231,0.003407,NaN,NaN
std,0.014825,0.008741,NaN,NaN
min,0.000149,0.000514,NaN,NaN
25%,0.000344,0.000558,NaN,NaN
50%,0.000426,0.000638,NaN,NaN
75%,0.000579,0.000767,NaN,NaN
max,0.047402,0.028283,NaN,NaN


In [20]:
metrics['TPU_14'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000524,0.000470,NaN,NaN
std,0.000531,0.000254,NaN,NaN
min,0.000076,0.000210,NaN,NaN
25%,0.000127,0.000271,NaN,NaN
50%,0.000306,0.000411,NaN,NaN
75%,0.000879,0.000533,NaN,NaN
max,0.001460,0.000955,NaN,NaN


In [21]:
metrics['TPU_15'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000679,0.000854,NaN,NaN
std,0.000999,0.000424,NaN,NaN
min,0.000034,0.000443,NaN,NaN
25%,0.000222,0.000587,NaN,NaN
50%,0.000325,0.000739,NaN,NaN
75%,0.000647,0.000932,NaN,NaN
max,0.003444,0.001891,NaN,NaN


In [22]:
metrics['TPU_16'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000611,0.000629,NaN,NaN
std,0.000730,0.000238,NaN,NaN
min,0.000174,0.000413,NaN,NaN
25%,0.000208,0.000479,NaN,NaN
50%,0.000317,0.000544,NaN,NaN
75%,0.000492,0.000725,NaN,NaN
max,0.002481,0.001191,NaN,NaN


In [14]:
metrics['TPU_17'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000692,0.001053,NaN,NaN
std,0.000395,0.000205,NaN,NaN
min,0.000213,0.000682,NaN,NaN
25%,0.000364,0.000982,NaN,NaN
50%,0.000617,0.001113,NaN,NaN
75%,0.000992,0.001177,NaN,NaN
max,0.001395,0.001334,NaN,NaN


In [29]:
metrics['TPU_18'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000519,0.000400,NaN,NaN
std,0.000559,0.000113,NaN,NaN
min,0.000036,0.000247,NaN,NaN
25%,0.000240,0.000313,NaN,NaN
50%,0.000394,0.000367,NaN,NaN
75%,0.000490,0.000482,NaN,NaN
max,0.002034,0.000577,NaN,NaN


In [31]:
metrics['TPU_19'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000566,0.000422,NaN,NaN
std,0.000657,0.000111,NaN,NaN
min,0.000047,0.000304,NaN,NaN
25%,0.000189,0.000358,NaN,NaN
50%,0.000401,0.000398,NaN,NaN
75%,0.000653,0.000440,NaN,NaN
max,0.002295,0.000703,NaN,NaN


In [33]:
metrics['TPU_20'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000247,0.000310,NaN,NaN
std,0.000262,0.000101,NaN,NaN
min,0.000041,0.000134,NaN,NaN
25%,0.000081,0.000262,NaN,NaN
50%,0.000181,0.000311,NaN,NaN
75%,0.000295,0.000333,NaN,NaN
max,0.000928,0.000531,NaN,NaN


In [36]:
metrics['TPU_21'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000524,0.000448,NaN,NaN
std,0.000327,0.000175,NaN,NaN
min,0.000155,0.000185,NaN,NaN
25%,0.000246,0.000321,NaN,NaN
50%,0.000440,0.000432,NaN,NaN
75%,0.000840,0.000582,NaN,NaN
max,0.000962,0.000728,NaN,NaN


In [12]:
metrics['TPU_22'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000444,0.000454,NaN,NaN
std,0.000376,0.000183,NaN,NaN
min,0.000020,0.000255,NaN,NaN
25%,0.000155,0.000324,NaN,NaN
50%,0.000352,0.000426,NaN,NaN
75%,0.000593,0.000519,NaN,NaN
max,0.001194,0.000849,NaN,NaN


In [11]:
metrics['TPU_23'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000380,0.000353,NaN,NaN
std,0.000368,0.000169,NaN,NaN
min,0.000039,0.000161,NaN,NaN
25%,0.000210,0.000212,NaN,NaN
50%,0.000322,0.000330,NaN,NaN
75%,0.000398,0.000446,NaN,NaN
max,0.001342,0.000645,NaN,NaN


In [15]:
metrics['TPU_24'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000289,0.000400,NaN,NaN
std,0.000236,0.000451,NaN,NaN
min,0.000032,0.000126,NaN,NaN
25%,0.000125,0.000192,NaN,NaN
50%,0.000260,0.000287,NaN,NaN
75%,0.000358,0.000302,NaN,NaN
max,0.000726,0.001651,NaN,NaN


In [17]:
metrics['TPU_25'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000248,0.000324,NaN,NaN
std,0.000158,0.000123,NaN,NaN
min,0.000072,0.000160,NaN,NaN
25%,0.000095,0.000243,NaN,NaN
50%,0.000256,0.000297,NaN,NaN
75%,0.000351,0.000399,NaN,NaN
max,0.000522,0.000567,NaN,NaN


In [19]:
metrics['TPU_26'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.002264,0.000386,NaN,NaN
std,0.005954,0.000096,NaN,NaN
min,0.000031,0.000281,NaN,NaN
25%,0.000080,0.000307,NaN,NaN
50%,0.000268,0.000365,NaN,NaN
75%,0.000653,0.000442,NaN,NaN
max,0.019166,0.000570,NaN,NaN


In [14]:
metrics['TPU_27'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000475,0.000303,NaN,NaN
std,0.000449,0.000129,NaN,NaN
min,0.000064,0.000125,NaN,NaN
25%,0.000154,0.000224,NaN,NaN
50%,0.000311,0.000270,NaN,NaN
75%,0.000808,0.000407,NaN,NaN
max,0.001328,0.000500,NaN,NaN


In [12]:
metrics['TPU_30'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000206,0.000253,NaN,NaN
std,0.000147,0.000043,NaN,NaN
min,0.000045,0.000173,NaN,NaN
25%,0.000107,0.000239,NaN,NaN
50%,0.000125,0.000268,NaN,NaN
75%,0.000334,0.000281,NaN,NaN
max,0.000436,0.000299,NaN,NaN


In [13]:
metrics['TPU_31'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000263,0.000363,NaN,NaN
std,0.000134,0.000107,NaN,NaN
min,0.000086,0.000197,NaN,NaN
25%,0.000175,0.000317,NaN,NaN
50%,0.000251,0.000361,NaN,NaN
75%,0.000310,0.000458,NaN,NaN
max,0.000552,0.000497,NaN,NaN


In [15]:
metrics['TPU_32'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000349,0.000411,NaN,NaN
std,0.000245,0.000113,NaN,NaN
min,0.000059,0.000280,NaN,NaN
25%,0.000223,0.000320,NaN,NaN
50%,0.000310,0.000390,NaN,NaN
75%,0.000372,0.000479,NaN,NaN
max,0.000865,0.000622,NaN,NaN


In [17]:
metrics['TPU_33'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000410,0.000382,NaN,NaN
std,0.000377,0.000068,NaN,NaN
min,0.000081,0.000235,NaN,NaN
25%,0.000169,0.000352,NaN,NaN
50%,0.000296,0.000384,NaN,NaN
75%,0.000506,0.000419,NaN,NaN
max,0.001344,0.000486,NaN,NaN


In [19]:
metrics['TPU_34'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000390,0.000274,NaN,NaN
std,0.000343,0.000142,NaN,NaN
min,0.000052,0.000159,NaN,NaN
25%,0.000114,0.000216,NaN,NaN
50%,0.000221,0.000223,NaN,NaN
75%,0.000651,0.000281,NaN,NaN
max,0.000954,0.000662,NaN,NaN


In [21]:
metrics['TPU_35'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.036321,0.017536,NaN,NaN
std,0.111247,0.051963,NaN,NaN
min,0.000356,0.000785,NaN,NaN
25%,0.000814,0.001002,NaN,NaN
50%,0.001157,0.001075,NaN,NaN
75%,0.001811,0.001153,NaN,NaN
max,0.352932,0.165423,NaN,NaN


In [14]:
metrics['TPU_36'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,9.000000,9.000000,0.0,0.0
mean,0.000388,0.000320,NaN,NaN
std,0.000254,0.000123,NaN,NaN
min,0.000062,0.000174,NaN,NaN
25%,0.000269,0.000202,NaN,NaN
50%,0.000329,0.000327,NaN,NaN
75%,0.000570,0.000367,NaN,NaN
max,0.000786,0.000538,NaN,NaN


In [18]:
metrics['TPU_37'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.024284,0.016817,NaN,NaN
std,0.074991,0.052215,NaN,NaN
min,0.000090,0.000156,NaN,NaN
25%,0.000317,0.000192,NaN,NaN
50%,0.000369,0.000359,NaN,NaN
75%,0.000667,0.000432,NaN,NaN
max,0.237707,0.165423,NaN,NaN


In [20]:
metrics['TPU_38'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000339,0.000289,NaN,NaN
std,0.000429,0.000174,NaN,NaN
min,0.000033,0.000124,NaN,NaN
25%,0.000106,0.000187,NaN,NaN
50%,0.000252,0.000259,NaN,NaN
75%,0.000333,0.000288,NaN,NaN
max,0.001510,0.000735,NaN,NaN


In [27]:
metrics['TPU_39'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000999,0.000490,NaN,NaN
std,0.001924,0.000558,NaN,NaN
min,0.000056,0.000179,NaN,NaN
25%,0.000106,0.000250,NaN,NaN
50%,0.000191,0.000326,NaN,NaN
75%,0.000687,0.000407,NaN,NaN
max,0.006269,0.002051,NaN,NaN


In [30]:
metrics['TPU_40'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000252,0.000253,NaN,NaN
std,0.000244,0.000128,NaN,NaN
min,0.000058,0.000097,NaN,NaN
25%,0.000131,0.000145,NaN,NaN
50%,0.000178,0.000243,NaN,NaN
75%,0.000255,0.000319,NaN,NaN
max,0.000889,0.000460,NaN,NaN


In [35]:
metrics['TPU_41'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.001860,0.002080,NaN,NaN
std,0.004652,0.005592,NaN,NaN
min,0.000036,0.000207,NaN,NaN
25%,0.000084,0.000220,NaN,NaN
50%,0.000306,0.000287,NaN,NaN
75%,0.000883,0.000370,NaN,NaN
max,0.015050,0.017991,NaN,NaN


In [37]:
metrics['TPU_42'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,10.000000,10.000000,0.0,0.0
mean,0.000184,0.000246,NaN,NaN
std,0.000107,0.000119,NaN,NaN
min,0.000018,0.000066,NaN,NaN
25%,0.000126,0.000165,NaN,NaN
50%,0.000166,0.000250,NaN,NaN
75%,0.000254,0.000334,NaN,NaN
max,0.000387,0.000433,NaN,NaN


In [10]:
metrics['TPU_50'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,30.000000,30.000000,0.0,0.0
mean,0.000273,0.000242,NaN,NaN
std,0.000227,0.000115,NaN,NaN
min,0.000045,0.000057,NaN,NaN
25%,0.000121,0.000151,NaN,NaN
50%,0.000204,0.000215,NaN,NaN
75%,0.000346,0.000326,NaN,NaN
max,0.000972,0.000480,NaN,NaN


In [11]:
metrics['TPU_51'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,30.000000,30.000000,0.0,0.0
mean,0.000268,0.000260,NaN,NaN
std,0.000184,0.000051,NaN,NaN
min,0.000032,0.000179,NaN,NaN
25%,0.000149,0.000224,NaN,NaN
50%,0.000220,0.000258,NaN,NaN
75%,0.000409,0.000298,NaN,NaN
max,0.000771,0.000358,NaN,NaN


In [13]:
metrics['TPU_52'].describe()

,train_value,eval_value,predict_mse,predict_mae
count,20.000000,20.000000,0.0,0.0
mean,0.000154,0.000108,NaN,NaN
std,0.000147,0.000029,NaN,NaN
min,0.000016,0.000064,NaN,NaN
25%,0.000052,0.000088,NaN,NaN
50%,0.000119,0.000109,NaN,NaN
75%,0.000196,0.000122,NaN,NaN
max,0.000628,0.000186,NaN,NaN


In [17]:
metrics['TPU_52']['eval_value'].mean()**.5

0.010398219544062308